- Author: Benjamin Du
- Date: 2022-12-30 17:45:15
- Modified: 2022-12-30 17:45:15
- Title: Memory Layout of Enum
- Slug: memory-layout-of-enum
- Category: Computer Science
- Tags: Computer Science, programming, Rust, Enum, memory, layout, heap, stack, tagged, union

**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

## Tips & Traps

1. The closest thing to describe Rust Enum is tagged Union.
    The Rust compiler adds an extra (up to) 8 bytes 
    to the enum to store the discriminator. 
    This is used to identify the variant currently stored in the enum.
    However, 
    Rust does NOT guarantee that the memory layout of an enum is always a tag followed by a union.
    Instead, 
    the Rust compiler retains the freedom to optimize the layout of types
    so that the code can be more memory efficient.

2. The stack size of an enum depends on the largest variant. 
    For example, 
    the following enum
    
        :::Rust
        enum MyEnum {
            A(Vec<u64>),
            B(isize),
            C(usize),
        }

    has the largest variant `Vec<usize>` which takes 24 bytes on stack.
    And since there's an extra 8 bytes for the tag,
    the total size of `MyEnum` on the stack is 32 bytes.
    However, 
    notice `Result<Vec<u64>, usize>` takes only 24 bytes on the stack 
    as the Rust compiler is able to optimize away the tag from the memory layout in this case.

In [11]:
:dep memuse = "0.2.1"

In [12]:
use memuse::DynamicUsage;

In [19]:
std::mem::size_of::<Vec<u64>>()

24

In [21]:
std::mem::size_of::<&Vec<u64>>()

8

In [22]:
std::mem::size_of::<&[u64]>()

16

In [13]:
type T1 = Result<Vec<u64>, usize>;
std::mem::size_of::<T1>()

24

In [16]:
let v1: T1 = Ok(Vec::with_capacity(10));
v1.dynamic_usage()

80

In [17]:
let v2: T1 = Err(100);
v2.dynamic_usage()

0

In [8]:
type T2 = Result<String, usize>;
std::mem::size_of::<T2>()

24

In [24]:
enum MyEnum {
    A(Vec<u64>),
    B(isize),
    C(usize),
}

std::mem::size_of::<MyEnum>()

32

## References

- [Rust Crate - memuse](https://crates.io/crates/memuse)

- [Optimized enum sizes in Rust](https://adeschamps.github.io/enum-size)

- [Rust enum-match code generation](https://www.eventhelix.com/rust/rust-to-assembly-enum-match/#:~:text=Memory%20layout%20of%20a%20Rust%20enum&text=The%20total%20memory%20needed%20for,currently%20saved%20in%20the%20enum.&text=Note%3A%20A%2064%2Dbit%20discriminator%20might%20seem%20wasteful%20here.)